# Define Running Mode

- 'full_dataset = True' to use the full data set. If 'full_dataset = True', then a data set containing only data one year of the hobbies sales in TX2. 
- 'save_results = True' to save the dataframe in m5_challenge\data\feature_engineering\
      


In [1]:
full_dataset = False
save_results = False

# Import Packages

In [2]:
import utils
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import time
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# Data Import and Preparation

In [3]:
# Import data
submission = pd.read_csv(
    f'{utils.get_m5_root_dir()}/data/input/sample_submission.csv')
if full_dataset:
    df_merged = pd.read_csv(
        f'{utils.get_m5_root_dir()}/data/preprocessed/preprocessed_input_data.csv'
    )
else:
    df_merged = pd.read_csv(
        f'{utils.get_m5_root_dir()}/data/preprocessed/tx2_hobbies_1year.csv')

# extract training and validation data (drop evaluation) since we are still in validation phase
df_merged = df_merged.loc[df_merged['data_type'] != 'evaluation']

# print top and bottom lines
df_merged.head(5).append(df_merged.tail(5))

,id,item_id,dept_id,cat_id,store_id,state_id,d,sale,date,wm_yr_wk,...,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,data_type
0,HOBBIES_1_001_TX_2_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,TX_2,TX,d_1547,0.0,2015-04-24,11512,...,2015,NaN,NaN,NaN,NaN,0,0,0,8.26,train
1,HOBBIES_1_002_TX_2_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,TX_2,TX,d_1547,1.0,2015-04-24,11512,...,2015,NaN,NaN,NaN,NaN,0,0,0,3.97,train
2,HOBBIES_1_003_TX_2_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,TX_2,TX,d_1547,0.0,2015-04-24,11512,...,2015,NaN,NaN,NaN,NaN,0,0,0,2.97,train
3,HOBBIES_1_004_TX_2_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,TX_2,TX,d_1547,0.0,2015-04-24,11512,...,2015,NaN,NaN,NaN,NaN,0,0,0,4.64,train
4,HOBBIES_1_005_TX_2_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,TX_2,TX,d_1547,0.0,2015-04-24,11512,...,2015,NaN,NaN,NaN,NaN,0,0,0,2.73,train
223170,HOBBIES_2_145_TX_2_validation,HOBBIES_2_145,HOBBIES_2,HOBBIES,TX_2,TX,d_1941,NaN,2016-05-22,11617,...,2016,NaN,NaN,NaN,NaN,0,0,0,2.88,validation
223171,HOBBIES_2_146_TX_2_validation,HOBBIES_2_146,HOBBIES_2,HOBBIES,TX_2,TX,d_1941,NaN,2016-05-22,11617,...,2016,NaN,NaN,NaN,NaN,0,0,0,1.97,validation
223172,HOBBIES_2_147_TX_2_validation,HOBBIES_2_147,HOBBIES_2,HOBBIES,TX_2,TX,d_1941,NaN,2016-05-22,11617,...,2016,NaN,NaN,NaN,NaN,0,0,0,0.97,validation
223173,HOBBIES_2_148_TX_2_validation,HOBBIES_2_148,HOBBIES_2,HOBBIES,TX_2,TX,d_1941,NaN,2016-05-22,11617,...,2016,NaN,NaN,NaN,NaN,0,0,0,0.88,validation
223174,HOBBIES_2_149_TX_2_validation,HOBBIES_2_149,HOBBIES_2,HOBBIES,TX_2,TX,d_1941,NaN,2016-05-22,11617,...,2016,NaN,NaN,NaN,NaN,0,0,0,0.97,validation


In [4]:
# downcast numerical values to reduce mem usage
df_merged = utils.reduce_mem_usage(df_merged)

Mem. usage of decreased to 27.46 Mb (32.8% reduction)


In [5]:
# Convert categorical features to integers as the categorical values cause problems when using large datasets

df_merged = utils.encode_categorical(df_merged, [
    "item_id", "dept_id", "cat_id", "store_id", "state_id", "event_name_1",
    "event_type_1", "event_name_2", "event_type_2", 'd'
])

df_merged.head(5)

Mem. usage of decreased to 14.69 Mb (28.9% reduction)


,id,item_id,dept_id,cat_id,store_id,state_id,d,sale,date,wm_yr_wk,...,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,data_type
0,HOBBIES_1_001_TX_2_validation,0,0,0,0,0,0,0.0,2015-04-24,11512,...,2015,13,1,0,0,0,0,0,8.26,train
1,HOBBIES_1_002_TX_2_validation,1,0,0,0,0,0,1.0,2015-04-24,11512,...,2015,13,1,0,0,0,0,0,3.97,train
2,HOBBIES_1_003_TX_2_validation,2,0,0,0,0,0,0.0,2015-04-24,11512,...,2015,13,1,0,0,0,0,0,2.97,train
3,HOBBIES_1_004_TX_2_validation,3,0,0,0,0,0,0.0,2015-04-24,11512,...,2015,13,1,0,0,0,0,0,4.64,train
4,HOBBIES_1_005_TX_2_validation,4,0,0,0,0,0,0.0,2015-04-24,11512,...,2015,13,1,0,0,0,0,0,2.73,train


# Feature Engineering

In [6]:
# sales data

# rolling mean and rolling std (weekly, monthly, quarterly, and half-year)
df_merged['rolling_mean_t28'] = df_merged.groupby(
    ['id'])['sale'].transform(lambda x: x.shift(28).rolling(30).mean())

df_merged['rolling_std_t28'] = df_merged.groupby(
    ['id'])['sale'].transform(lambda x: x.shift(28).rolling(30).std())

df_merged['rolling_kurt_t28'] = df_merged.groupby(
    ['id'])['sale'].transform(lambda x: x.shift(28).rolling(28).kurt())

df_merged['rolling_skew_t28'] = df_merged.groupby(
    ['id'])['sale'].transform(lambda x: x.shift(28).rolling(30).skew())

df_merged['lag_t28'] = df_merged.groupby(
    ['id'])['sale'].transform(lambda x: x.shift(28))

# product was up for sale if price is not zero
df_merged['up_for_sale'] = np.where(df_merged['sell_price'].isna(), 0, 1)

In [7]:
# price data

df_merged['lag_price_t1'] = df_merged.groupby(
    ['id'])['sell_price'].transform(lambda x: x.shift(1))

df_merged['rolling_price_max_t30'] = df_merged.groupby(
    ['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(30).max())

df_merged['price_change_t1'] = (df_merged['lag_price_t1'] -
                                df_merged['sell_price']) / (
                                    df_merged['lag_price_t1'])

df_merged['price_change_t30'] = (df_merged['rolling_price_max_t30'] -
                                 df_merged['sell_price']) / (
                                     df_merged['rolling_price_max_t30'])

df_merged['rolling_price_std_t28'] = df_merged.groupby(
    ['id'])['sell_price'].transform(lambda x: x.rolling(28).std())

df_merged.drop(['rolling_price_max_t30', 'lag_price_t1'], inplace=True, axis=1)

In [8]:
# date data

# Saturday: wday = 1, Sunday: wday = 2
df_merged["is_weekend"] = df_merged["wday"].isin([1, 2]).astype(np.int8)

df_merged.loc[:, 'date'] = pd.to_datetime(df_merged['date'])
df_merged['day'] = df_merged['date'].dt.day.astype(np.int8)

In [9]:
# TODO: proper feature selection (e.g. random forrest)
# for now: hardcoded list of features

cat_features = [
    'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'year', 'month',
    'day', 'is_weekend', 'wday', 'event_name_1', 'event_type_1',
    'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI',
    'up_for_sale'
]

num_features = [
    'sell_price',
    'lag_t28',
    'rolling_kurt_t28',
    'rolling_skew_t28',
    'rolling_mean_t28',
    'rolling_std_t28',
    'price_change_t1',
    'price_change_t30',
    'rolling_price_std_t28',
]

features = cat_features + num_features

In [10]:
# drop first 180 days since they have missing caused by the feature engineering
date_after_90_training_days = str(df_merged['date'].dt.date.min() +
                                  pd.to_timedelta(180, unit='d'))
df_merged = df_merged[df_merged['date'] > date_after_90_training_days]

# Prepare Data for Training

In [11]:
x_train = df_merged[df_merged['date'] <= '2016-03-27'][features]
y_train = df_merged[df_merged['date'] <= '2016-03-27']['sale']

x_val = df_merged.loc[df_merged['data_type'] == 'train'].loc[
    df_merged['date'] > '2016-03-27'][features]
y_val = df_merged.loc[df_merged['data_type'] == 'train'].loc[
    df_merged['date'] > '2016-03-27']['sale']

x_pred = df_merged[(df_merged['data_type'] == 'validation')][features]

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_pred.shape)

(89270, 27)
(89270,)
(15820, 27)
(15820,)
(15820, 27)


In [12]:
# create datasets
train_data = lgb.Dataset(x_train,
                         label=y_train,
                         categorical_feature=cat_features,
                         free_raw_data=False)
validation_data = lgb.Dataset(x_val,
                              label=y_val,
                              categorical_feature=cat_features,
                              free_raw_data=False,
                              reference=train_data)

# Save Datasets

In [13]:
if save_results:

    if full_dataset:
        prefix = 'full_dataset'
    else:
        prefix = 'subset'

    train_data.save_binary(
        f'{utils.get_m5_root_dir()}/data/feature_engineering/{prefix}_train_lightgbm.bin'
    )
    validation_data.save_binary(
        f'{utils.get_m5_root_dir()}/data/feature_engineering/{prefix}_validation_data_lightgbm.bin'
    )

    df_merged.to_csv(
        f'{utils.get_m5_root_dir()}/data/feature_engineering/{prefix}_df_merged.csv',
        index=False)

    x_train.to_csv(
        f'{utils.get_m5_root_dir()}/data/feature_engineering/{prefix}_x_train.csv',
        index=False)
    y_train.to_csv(
        f'{utils.get_m5_root_dir()}/data/feature_engineering/{prefix}_y_train.csv',
        index=False)

    x_val.to_csv(
        f'{utils.get_m5_root_dir()}/data/feature_engineering/{prefix}_x_val.csv',
        index=False)
    y_val.to_csv(
        f'{utils.get_m5_root_dir()}/data/feature_engineering/{prefix}_y_val.csv',
        index=False)

    x_pred.to_csv(
        f'{utils.get_m5_root_dir()}/data/feature_engineering/{prefix}_x_pred.csv',
        index=False)